In [ ]:
! pip install scikit-learn pandas numpy matplotlib seaborn fireducks

In [1]:
from sklearn.datasets import load_diabetes
import fireducks.pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
diabetes = load_diabetes()

In [3]:
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])

In [4]:
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

In [5]:
X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [6]:
y = pd.Series(diabetes.target)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
lr = LinearRegression()

In [9]:
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
predict = lr.predict(X_test)

In [11]:
root_mean_squared_error(y_pred=predict, y_true=y_test)

53.12015607094269

In [12]:
mean_absolute_error(y_pred=predict, y_true=y_test)

41.919378456792735

### Algoritmo Genético

In [13]:
# Função de Aptidão
def aptidao(y_pred, y_true):
    return mean_absolute_error(y_pred=y_pred, y_true=y_true)

In [14]:
def set_random_state(feature:str) -> int:
    binary_value = np.random.randint(0, 2)
    return binary_value

In [15]:
# Função de inicialização
def init_individual(features: list) -> dict:
    individual = {}
    for feature in features:
        individual[feature] = set_random_state(feature)
    return individual

In [16]:
def init_population(features: list, n_individuals: int) -> list:
    population = []
    for i in range(n_individuals):
        individual = init_individual(features)
        population.append({f'indiv_{i}':individual})
    return population

In [17]:
def sub_set_features(ga: dict) -> list:
    sub_set = []
    for feature, value in ga.items():
        if value == 1:
            sub_set.append(feature)
    return sub_set

In [18]:
def test_sub_set_features(model, X_train_set, X_test_set, y_train, y_test):
    model.fit(X_train_set, y_train)
    values_predict = model.predict(X_test_set)
    evaluation = aptidao(y_pred=values_predict, y_true=y_test)
    return evaluation

In [19]:
def set_mutation(individual_set: dict) -> dict:
    random_gene = np.random.choice(list(individual_set.keys()))
    if individual_set[random_gene] == 1:
        individual_set[random_gene] = 0
    else:
        individual_set[random_gene] = 1
    return individual_set

In [20]:
pop_test = [{'indiv_0': {'age': 1, 'sex': 0, 'bmi': 1, 'bp': 1, 's1': 1, 's2': 1, 's3': 1, 's4': 1, 's5': 0, 's6': 1, 'fitness': 43.77931136571283}}, {'indiv_1': {'age': 0, 'sex': 1, 'bmi': 0, 'bp': 1, 's1': 1, 's2': 0, 's3': 0, 's4': 1, 's5': 0, 's6': 1, 'fitness': 49.631346257072664}}, {'indiv_2': {'age': 0, 'sex': 0, 'bmi': 0, 'bp': 1, 's1': 1, 's2': 1, 's3': 1, 's4': 0, 's5': 1, 's6': 0, 'fitness': 45.631519756695475}}]

In [21]:
def best_fitness(population: list) -> dict:
    best_fitness = [val['fitness'] for population in pop_test for val in population.values()]
    return max(best_fitness)

In [22]:
def best_individual(population: list) -> list:
    for individual in population:
        for val in individual.values():
            if val['fitness'] == best_fitness(population):
                val.pop('fitness')
                return val

In [23]:
def best_features(population: list) -> list:
    best_features = []
    for key, value in best_individual(population).items():
        if value == 1:
            best_features.append(key)
    return best_features

In [24]:
def evaluate_population(population: list, model, X_train, X_test, y_train, y_test) -> list:
    for individual in population:
        for key, value in individual.items():
            sub_set = sub_set_features(value)
            evaluation = test_sub_set_features(model, X_train[sub_set], X_test[sub_set], y_train, y_test)
            value['fitness'] = evaluation
    return population

**Etapas de um Algoritmo Genético**

Um algoritmo genético (GA) é um algoritmo de busca e otimização inspirado no processo de seleção natural e genética. As etapas principais de um GA são:

1.  **Inicialização:**

    *   Cria-se uma população inicial de soluções candidatas (indivíduos).
    *   Cada indivíduo é representado por um cromossomo, que é uma sequência de genes que codificam uma possível solução para o problema.
    *   A população inicial pode ser gerada aleatoriamente ou utilizando algum conhecimento prévio sobre o problema.
2.  **Avaliação (Função de Aptidão):**

    *   Cada indivíduo na população é avaliado utilizando uma função de aptidão (fitness function).
    *   A função de aptidão quantifica a qualidade da solução representada pelo indivíduo.
    *   Indivíduos com maior aptidão representam soluções melhores para o problema.
3.  **Seleção:**

    *   Seleciona-se um subconjunto de indivíduos da população para reprodução.
    *   A seleção é baseada na aptidão dos indivíduos, de forma que indivíduos mais aptos têm maior probabilidade de serem selecionados.
    *   Métodos de seleção comuns incluem:
        *   **Seleção por Roleta:** A probabilidade de seleção é proporcional à aptidão.
        *   **Seleção por Torneio:** Seleciona-se um grupo de indivíduos aleatoriamente e o mais apto é escolhido.
        *   **Seleção por Ranking:** Os indivíduos são classificados por aptidão e a seleção é baseada na sua posição no ranking.
4.  **Crossover (Recombinação):**

    *   Os indivíduos selecionados (pais) são combinados para gerar novos indivíduos (filhos).
    *   O crossover simula a reprodução sexual, onde os genes dos pais são combinados para formar a prole.
    *   Métodos de crossover comuns incluem:
        *   **Crossover de um ponto:** Um ponto é escolhido aleatoriamente e os genes dos pais são trocados a partir desse ponto.
        *   **Crossover de dois pontos:** Dois pontos são escolhidos aleatoriamente e os genes entre esses pontos são trocados.
        *   **Crossover uniforme:** Cada gene é selecionado aleatoriamente de um dos pais.
5.  **Mutação:**

    *   Após o crossover, aplica-se uma mutação a alguns dos filhos.
    *   A mutação introduz pequenas alterações aleatórias nos genes dos indivíduos.
    *   A mutação ajuda a manter a diversidade na população e evita a convergência prematura para soluções subótimas.
    *   A taxa de mutação é geralmente baixa.
6.  **Substituição:**

    *   A nova geração de filhos substitui a geração antiga de pais.
    *   Métodos de substituição comuns incluem:
        *   **Substituição completa:** Toda a população antiga é substituída pela nova.
        *   **Substituição parcial:** Apenas alguns dos indivíduos da população antiga são substituídos pelos novos.
        *   **Elitismo:** Os melhores indivíduos da população antiga são mantidos na nova geração.
7.  **Critério de Parada:**

    *   O algoritmo genético é repetido (avaliação, seleção, crossover, mutação, substituição) até que um critério de parada seja satisfeito.
    *   Critérios de parada comuns incluem:
        *   **Número máximo de gerações:** O algoritmo é executado por um número fixo de gerações.
        *   **Convergência:** A população converge para uma solução, ou seja, a aptidão dos melhores indivíduos não melhora significativamente ao longo das gerações.
        *   **Aptidão satisfatória:** Uma solução com aptidão suficiente é encontrada.

In [25]:
def ga_app(num_pop_init: int, features:list, num_generations: int, lr, X_train, X_test, y_train, y_test) -> dict:
    # Inicialização da população
    population = init_population(features, num_pop_init)
    print("POP INIT", population)
    
    # Avaliação da população
    population_evaluated = evaluate_population(population, lr, X_train, X_test, y_train, y_test)
    print("POPULATION EVALUATED", population_evaluated)

    # Melhor indivíduo
    best_individual(population_evaluated)
    print("BEST INDIVIDUAL", best_individual(population_evaluated))

    # Melhor fitness
    best_fitness(population_evaluated)
    print("BEST FITNESS", best_fitness(population_evaluated))

    # Melhor conjunto de features
    best_features(population_evaluated)
    print("BEST FEATURES", best_features(population_evaluated))


In [26]:
ga_app(num_pop_init=3, features= list(X.columns), num_generations=7, lr=lr, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

POP INIT [{'indiv_0': {'age': 1, 'sex': 0, 'bmi': 0, 'bp': 0, 's1': 1, 's2': 0, 's3': 1, 's4': 0, 's5': 0, 's6': 1}}, {'indiv_1': {'age': 1, 'sex': 1, 'bmi': 0, 'bp': 0, 's1': 1, 's2': 0, 's3': 0, 's4': 0, 's5': 1, 's6': 1}}, {'indiv_2': {'age': 0, 'sex': 0, 'bmi': 0, 'bp': 1, 's1': 1, 's2': 1, 's3': 0, 's4': 1, 's5': 0, 's6': 1}}]
POPULATION EVALUATED [{'indiv_0': {'age': 1, 'sex': 0, 'bmi': 0, 'bp': 0, 's1': 1, 's2': 0, 's3': 1, 's4': 0, 's5': 0, 's6': 1, 'fitness': 52.48350564908008}}, {'indiv_1': {'age': 1, 'sex': 1, 'bmi': 0, 'bp': 0, 's1': 1, 's2': 0, 's3': 0, 's4': 0, 's5': 1, 's6': 1, 'fitness': 44.85684142768842}}, {'indiv_2': {'age': 0, 'sex': 0, 'bmi': 0, 'bp': 1, 's1': 1, 's2': 1, 's3': 0, 's4': 1, 's5': 0, 's6': 1, 'fitness': 49.71516240190114}}]
BEST INDIVIDUAL None
BEST FITNESS 49.631346257072664


AttributeError: 'NoneType' object has no attribute 'items'